<a href="https://colab.research.google.com/github/KLT20/Kaggle_Competition_Winter2021/blob/main/Lea_Kassab_Kaggle_Competition_MAIS202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [7]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [8]:
!kaggle competitions download -c mais-202-winter-2021-kaggle-comp

train_y.csv: Skipping, found more recently modified local copy (use --force to force download)
 96% 189M/196M [00:03<00:00, 82.8MB/s]
100% 196M/196M [00:03<00:00, 64.5MB/s]
 99% 778M/784M [00:08<00:00, 99.1MB/s]
100% 784M/784M [00:09<00:00, 90.9MB/s]


In [9]:
!unzip \*.zip  && rm *.zip

Archive:  test_x.npy.zip
replace test_x.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_x.npy              

Archive:  train_x.npy.zip
  inflating: train_x.npy             A


2 archives were successfully processed.


The step-wise process to *download the data from Kaggle* was found at the **following link**: https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a 

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

Extremely helpful ressource for the Plain_Dataset class: https://debuggercafe.com/custom-dataset-and-dataloader-in-pytorch/ 

In [11]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()


In [12]:
#load the data:
trainset = np.load("train_x.npy")
testset = np.load("test_x.npy")
df_train = pd.read_csv("/content/gdrive/My Drive/Kaggle/train_y.csv")

#takes the training labels here
train_labels = df_train.iloc[:, 1]
#takes the training images here
train_images = trainset[:]
#takes the test images here
test_images = testset[:]




In [13]:
#create empty array for test_labels
test_labels = np.empty(10000, dtype=object) 

In [14]:
# we convert the data into PIL image, then to PyTorch tensors, and finally, we normalize the image data. 
transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])

In [19]:
class LeaCalssifier(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        #data = self.X.iloc[i, :] #might need to just remove iloc
        data = self.X[i, :]
        data = np.asarray(data).astype(np.uint8).reshape(128, 128, 1) #because size of image is 128x128

        if self.transforms:
          data = self.transforms(data) #if transform define, convert to images
        
        if self.y is not None:
          return (data, self.y[i]) #if labels exist then return them when called
        else: #this is for when we are loading the test set, since we dont have the labels
          return data #if they dont exist, return the image transfomed 

train_data = LeaCalssifier(train_images, train_labels, transform)

#test_labels dont exist because we are trying to predict them
test_data = LeaCalssifier(test_images, test_labels, transform)
# dataloaders
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)
testloader = DataLoader(test_data, batch_size=128, shuffle=True)


In [48]:
# print(len(trainset[1]))
# print("here is the next one ---------------------------")
# df_train = pd.read_csv("/content/gdrive/My Drive/Kaggle/train_y.csv")
# print(trainset[3])
# print("if these are the as next")
# print(df_train.at[4,'Label'])
# print(df_train.at[5,'Label'])
# print(df_train.at[6,'Label'])
# print("if same we good")
# print(df_train.iloc[:6, 1])


Here, I created the CNN with the understanding from the following youtube video: https://www.youtube.com/watch?v=LgFNRIFxuUo&ab_channel=SungKim 

In [28]:
import torch
import torch.nn as nn

class LeaCNN(nn.Module):
    def __init__(self):
        super(LeaCNN, self).__init__()
        # 128x128x1 is input of Linear since our pictures are 128px in height and 128px in width
        # and we set the in_channel to 1 since it's a grayscale picture
        # set arbitrary output channel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        
        #maxpooling
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)        
        self.pool3 = nn.MaxPool2d(2,2)

        #activation function
        # linear's input is the size of the image (128*128) multiplied by the output channels
        # output is 10 because we have 10 possible classification of the mnist digits (from 0 to 9)
        self.fc = nn.Linear( 1960 , 10)

    def forward(self, x):    
        x = self.conv1(x)
        x = self.pool1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.pool2(x)
        x = F.relu(x)
        
        x = self.conv3(x)
        x = self.pool3(x)
        x = F.relu(x)

        x = x.view(x.size(0), -1)  #flatten the tensor
        x = self.fc(x)

        return x

Next, instantiating a model starts by defining an instance `model`, a loss function module `criterion` and optimizer `optimizer`. This adjusts the parameters of our model to minimize the loss output by `criterion`.

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LeaCNN()
model = model.to(device)
#loss
criterion = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Training function that will be executed that I implemented by understanding the following link: https://debuggercafe.com/custom-dataset-and-dataloader-in-pytorch/


In [35]:
def train(model, trainloader):
    for epoch in range(20): # no. of epochs
        running_loss = 0
        for data in trainloader:
            # data pixels and labels to GPU if available
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            # set the parameter gradients to zero
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # propagate the loss backward
            loss.backward()
            # update the gradients
            optimizer.step()
 
            running_loss += loss.item()
        print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss/len(trainloader)))
 
    print('Done Training')


In [36]:
def test(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on test images: %0.3f %%' % (100 * correct / total))


Run the test and train functions to see results:

In [40]:
train(model, trainloader)

KeyboardInterrupt: ignored

In [41]:
test(model,testloader)     

TypeError: ignored